# Transfer Learning Model 'Xception' #

In [1]:

import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from keras.applications.xception import Xception


Using TensorFlow backend.


In [ ]:
def get_pcam_generators(base_dir, train_batch_size=32, val_batch_size=32):

     # dataset parameters
     train_path = os.path.join(base_dir, 'train+val','train')
     valid_path = os.path.join(base_dir, 'train+val','valid')

     # instantiate data generators
     datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

     train_gen = datagen.flow_from_directory(train_path,
                                             target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                             batch_size=train_batch_size,
                                             class_mode='binary')

     val_gen = datagen.flow_from_directory(valid_path,
                                             target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                             batch_size=val_batch_size,
                                             class_mode='binary')

     return train_gen, val_gen

# the size of the images in the PCAM dataset
IMAGE_SIZE = 96

train_gen, val_gen = get_pcam_generators('C:/Users/Daniel/Documents/')
input_shape = (IMAGE_SIZE, IMAGE_SIZE, 3)
input = Input(input_shape)
# get the pretrained model, cut out the top layer
pretrained = Xception(input_shape=input_shape, include_top=False, weights='imagenet')

layernames = []
for i,layer in enumerate(pretrained.layers):
    layernames.append(layer.name)

Found 144000 images belonging to 2 classes.
Found 16000 images belonging to 2 classes.


In [ ]:
for j in range(14,0,-1): #verander hier het aantal blokken dat je pretrained model heeft
    input_shape = (IMAGE_SIZE, IMAGE_SIZE, 3)
    input = Input(input_shape)
    pretrained = Xception(input_shape=input_shape, include_top=False, weights='imagenet')
    for layer in pretrained.layers:
            layer.trainable = False            

    string = 'block'+str(j)
    indices = [i for i, s in enumerate(layernames) if string in s]      
    for layer in pretrained.layers[min(indices):]:
        layer.trainable = True
        
    output = pretrained(input)
    output = GlobalAveragePooling2D()(output)
    output = Dropout(0.5)(output)
    output = Dense(1, activation='sigmoid')(output)

    # Check the trainable status of the individual layers
    #for layer in pretrained.layers:
    #    print(layer, layer.trainable)

    model = Model(input, output)
    model.compile(SGD(lr=0.001, momentum=0.95), loss = 'binary_crossentropy', metrics=['accuracy'])
    #
    # print a summary of the model on screen
    print(string)
    for i,layer in enumerate(pretrained.layers):
        print(i,layer.name,layer.trainable)
    model.summary()
        
    # save the model and weights
    model_name = 'Xception2 '+string
    model_filepath = model_name + '.json'
    weights_filepath = model_name + '_weights.hdf5'

    model_json = model.to_json() # serialize model to JSON
    with open(model_filepath, 'w') as json_file:
        json_file.write(model_json)


    # define the model checkpoint and Tensorboard callbacks
    checkpoint = ModelCheckpoint(weights_filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
    tensorboard = TensorBoard(os.path.join('logs', model_name))
    callbacks_list = [checkpoint, tensorboard]


    # train the model, note that we define "mini-epochs"
    train_steps = train_gen.n//train_gen.batch_size//20
    val_steps = val_gen.n//val_gen.batch_size//20

    # since the model is trained for only 10 "mini-epochs", i.e. half of the data is
    # not used during training

    history = model.fit_generator(train_gen, steps_per_epoch=train_steps,
                        validation_data=val_gen,
                        validation_steps=val_steps,
                        epochs=10,
                        callbacks=callbacks_list)



block14
0 input_4 False
1 block1_conv1 False
2 block1_conv1_bn False
3 block1_conv1_act False
4 block1_conv2 False
5 block1_conv2_bn False
6 block1_conv2_act False
7 block2_sepconv1 False
8 block2_sepconv1_bn False
9 block2_sepconv2_act False
10 block2_sepconv2 False
11 block2_sepconv2_bn False
12 conv2d_5 False
13 block2_pool False
14 batch_normalization_5 False
15 add_13 False
16 block3_sepconv1_act False
17 block3_sepconv1 False
18 block3_sepconv1_bn False
19 block3_sepconv2_act False
20 block3_sepconv2 False
21 block3_sepconv2_bn False
22 conv2d_6 False
23 block3_pool False
24 batch_normalization_6 False
25 add_14 False
26 block4_sepconv1_act False
27 block4_sepconv1 False
28 block4_sepconv1_bn False
29 block4_sepconv2_act False
30 block4_sepconv2 False
31 block4_sepconv2_bn False
32 conv2d_7 False
33 block4_pool False
34 batch_normalization_7 False
35 add_15 False
36 block5_sepconv1_act False
37 block5_sepconv1 False
38 block5_sepconv1_bn False
39 block5_sepconv2_act False
40 bloc

Epoch 1/10
225/225 [==============================] - 40s 178ms/step - loss: 0.5880 - acc: 0.6928 - val_loss: 1.2215 - val_acc: 0.5925

Epoch 00001: val_loss improved from inf to 1.22154, saving model to Xception2 block13_weights.hdf5
Epoch 2/10
225/225 [==============================] - 35s 158ms/step - loss: 0.4535 - acc: 0.7953 - val_loss: 1.2136 - val_acc: 0.6125

Epoch 00002: val_loss improved from 1.22154 to 1.21365, saving model to Xception2 block13_weights.hdf5
Epoch 3/10
225/225 [==============================] - 36s 158ms/step - loss: 0.4239 - acc: 0.8053 - val_loss: 1.3782 - val_acc: 0.6162

Epoch 00003: val_loss did not improve from 1.21365
Epoch 4/10
225/225 [==============================] - 36s 159ms/step - loss: 0.4027 - acc: 0.8215 - val_loss: 1.0459 - val_acc: 0.6000

Epoch 00004: val_loss improved from 1.21365 to 1.04591, saving model to Xception2 block13_weights.hdf5
Epoch 5/10
225/225 [==============================] - 36s 159ms/step - loss: 0.3836 - acc: 0.8289 - 

Epoch 9/10
225/225 [==============================] - 41s 182ms/step - loss: 0.3420 - acc: 0.8488 - val_loss: 0.5412 - val_acc: 0.7312

Epoch 00009: val_loss did not improve from 0.53155
Epoch 10/10
225/225 [==============================] - 41s 182ms/step - loss: 0.3506 - acc: 0.8525 - val_loss: 0.5751 - val_acc: 0.6863

Epoch 00010: val_loss did not improve from 0.53155
block11
0 input_10 False
1 block1_conv1 False
2 block1_conv1_bn False
3 block1_conv1_act False
4 block1_conv2 False
5 block1_conv2_bn False
6 block1_conv2_act False
7 block2_sepconv1 False
8 block2_sepconv1_bn False
9 block2_sepconv2_act False
10 block2_sepconv2 False
11 block2_sepconv2_bn False
12 conv2d_17 False
13 block2_pool False
14 batch_normalization_17 False
15 add_49 False
16 block3_sepconv1_act False
17 block3_sepconv1 False
18 block3_sepconv1_bn False
19 block3_sepconv2_act False
20 block3_sepconv2 False
21 block3_sepconv2_bn False
22 conv2d_18 False
23 block3_pool False
24 batch_normalization_18 False
25 a

Epoch 1/10
225/225 [==============================] - 54s 239ms/step - loss: 0.5743 - acc: 0.7032 - val_loss: 0.6326 - val_acc: 0.6212

Epoch 00001: val_loss improved from inf to 0.63256, saving model to Xception2 block10_weights.hdf5
Epoch 2/10
225/225 [==============================] - 51s 226ms/step - loss: 0.4279 - acc: 0.8086 - val_loss: 0.6309 - val_acc: 0.6600

Epoch 00002: val_loss improved from 0.63256 to 0.63090, saving model to Xception2 block10_weights.hdf5
Epoch 3/10
225/225 [==============================] - 51s 227ms/step - loss: 0.3878 - acc: 0.8244 - val_loss: 0.5945 - val_acc: 0.6925

Epoch 00003: val_loss improved from 0.63090 to 0.59447, saving model to Xception2 block10_weights.hdf5
Epoch 4/10
225/225 [==============================] - 51s 227ms/step - loss: 0.3818 - acc: 0.8318 - val_loss: 0.5259 - val_acc: 0.7188

Epoch 00004: val_loss improved from 0.59447 to 0.52595, saving model to Xception2 block10_weights.hdf5
Epoch 5/10
225/225 [============================

Epoch 8/10
225/225 [==============================] - 56s 249ms/step - loss: 0.3522 - acc: 0.8458 - val_loss: 0.5017 - val_acc: 0.7575

Epoch 00008: val_loss improved from 0.51398 to 0.50172, saving model to Xception2 block9_weights.hdf5
Epoch 9/10
225/225 [==============================] - 56s 249ms/step - loss: 0.3437 - acc: 0.8492 - val_loss: 0.5102 - val_acc: 0.7362

Epoch 00009: val_loss did not improve from 0.50172
Epoch 10/10
225/225 [==============================] - 56s 249ms/step - loss: 0.3246 - acc: 0.8586 - val_loss: 0.5046 - val_acc: 0.7588

Epoch 00010: val_loss did not improve from 0.50172
block8
0 input_16 False
1 block1_conv1 False
2 block1_conv1_bn False
3 block1_conv1_act False
4 block1_conv2 False
5 block1_conv2_bn False
6 block1_conv2_act False
7 block2_sepconv1 False
8 block2_sepconv1_bn False
9 block2_sepconv2_act False
10 block2_sepconv2 False
11 block2_sepconv2_bn False
12 conv2d_29 False
13 block2_pool False
14 batch_normalization_29 False
15 add_85 False
16 

Epoch 1/10
225/225 [==============================] - 69s 308ms/step - loss: 0.5858 - acc: 0.6886 - val_loss: 0.6187 - val_acc: 0.6412

Epoch 00001: val_loss improved from inf to 0.61865, saving model to Xception2 block7_weights.hdf5
Epoch 2/10
225/225 [==============================] - 65s 291ms/step - loss: 0.4480 - acc: 0.7937 - val_loss: 0.6359 - val_acc: 0.6500

Epoch 00002: val_loss did not improve from 0.61865
Epoch 3/10
225/225 [==============================] - 66s 292ms/step - loss: 0.4021 - acc: 0.8224 - val_loss: 0.7699 - val_acc: 0.6150

Epoch 00003: val_loss did not improve from 0.61865
Epoch 4/10
225/225 [==============================] - 66s 292ms/step - loss: 0.3886 - acc: 0.8268 - val_loss: 0.5560 - val_acc: 0.7025

Epoch 00004: val_loss improved from 0.61865 to 0.55597, saving model to Xception2 block7_weights.hdf5
Epoch 5/10
225/225 [==============================] - 66s 291ms/step - loss: 0.3592 - acc: 0.8401 - val_loss: 0.6210 - val_acc: 0.6687

Epoch 00005: val_l